<a href="https://colab.research.google.com/github/wellingtonsuper17/Atv1_poo/blob/main/Ufal_Copeve_Sisu_G%C3%AAnero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Initialization

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime

In [5]:
def get_relatorio_inicial(periodo):
    url = f"https://copeve.ufal.br/concursos/sisu_{periodo}_acompanhamento.php"
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
    }
    resposta = requests.get(url, headers=headers)
    if resposta.status_code != 200:
      return False
    relatorio = BeautifulSoup(resposta.text, 'html.parser')
    return relatorio


def get_demandas(bs):
    select = bs.find('select', {'name': 'demanda', 'class': 'registro'})
    demandas  = []
    for option in select.find_all('option'):
        demandas.append((option['value'], option.text))
    return demandas

def get_cursos(bs):
    select = bs.find('select', {'name': 'idCurso', 'class': 'registro'})
    cursos = []
    for option in select.find_all('option'):
        if 'Engenharia De Computação' in option.text or 'Ciência Da Computação' in option.text or 'Sistemas De Informação' in option.text:
            cursos.append((option['value'], option.text))
    return cursos

In [6]:
def get_relatorio_final(periodo, idCurso, demanda):
    url = f"https://copeve.ufal.br/concursos/sisu_{periodo}/resultado_curso.php"
    headers = {
      "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
      "Accept-Encoding": "gzip, deflate, br",
      "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
      "Cache-Control": "max-age=0",
      "Connection": "keep-alive",
      "Content-Type": "application/x-www-form-urlencoded",
      "Host": "copeve.ufal.br",
      "Origin": "https://copeve.ufal.br",
      "Referer": f"https://copeve.ufal.br/concursos/sisu_{periodo}_acompanhamento.php",
      "Sec-Fetch-Dest": "iframe",
      "Sec-Fetch-Mode": "navigate",
      "Sec-Fetch-Site": "same-origin",
      "Sec-Fetch-User": "?1",
      "Upgrade-Insecure-Requests": "1",
      "User-Agent": "Mozilla/5.0 (Windows NT 10...",
    }
    data = {
      'idCurso': idCurso,
      'demanda': demanda,
      'procurar': 'Consultar'
    }
    resposta = requests.post(url, headers=headers, data=data)
    if resposta.status_code != 200:
      return False
    relatorio = BeautifulSoup(resposta.text, 'html.parser')
    return relatorio

def bs_to_dataframe(bs):
  table = bs.find_all('table')[1]
  df = pd.read_html(str(table), decimal=',', header=1)[0]
  return df


In [7]:
import csv
import gzip
import io

def load_data():
    fobj = io.TextIOWrapper(gzip.open("nomes.csv.gz"), encoding="utf-8")
    csv_reader = csv.DictReader(fobj)
    data = {
        row["first_name"]: row["classification"]
        for row in csv_reader
    }
    fobj.close()
    return data

name_data = load_data()

In [8]:
from unicodedata import normalize

def encode(name):
    ascii_name = normalize("NFKD", name).encode("ascii", errors="ignore").decode("ascii")
    return ascii_name.upper()

def classify_sex(name):
    erros = []
    encoded_name = encode(name)
    try:
      sex = name_data[encoded_name]
      return sex
    except KeyError:
      return None

In [9]:
def gerar_periodos(ano_inicial):
    ano_atual = datetime.now().year
    periodos = []
    for ano in range(ano_inicial, ano_atual + 1):
        periodos.append(f'{ano}')
        periodos.append(f'{ano}_2')
    return periodos

def gerar_sexo(df):
    df['Sexo'] = ''
    for index, item in df['Nome'].iteritems():
        if isinstance(item, str):
          primeiro_nome = item.split(' ')[0]
        df.at[index, 'Sexo'] = classify_sex(primeiro_nome)
    return df

def get_relatorio_periodo(periodo, curso):
  relatorio = get_relatorio_inicial(periodo)
  demandas = get_demandas(relatorio)
  df_final = pd.DataFrame()
  for demanda in demandas:
    relatorio = get_relatorio_final(periodo, curso, demanda[0])
    if not relatorio:
      exit()
    df = bs_to_dataframe(relatorio)
    df = gerar_sexo(df)
    df_final = pd.concat([df_final, df])
  return df_final

def get_relatorio_demanda(periodo, curso, demanda):
  relatorio = get_relatorio_final(periodo, curso, demanda)
  if not relatorio:
    exit()
  df = bs_to_dataframe(relatorio)
  df = gerar_sexo(df)
  return df

# Loading and updating data

# Analysis

In [10]:
relatorio = get_relatorio_inicial('2022_2')
if relatorio:
  cursos = get_cursos(relatorio)
  for curso in cursos:
    print(curso)
  demandas = get_demandas(relatorio)
  for demanda in demandas:
    print(demanda)


('4', '004 - Ciência Da Computação - Bach - Vesp - A. C. Simões')
('18', '018 - Engenharia De Computação - Bach - Vesp - A. C. Simões')
('61', '061 - Sistemas De Informação - Bach - Not - C. Arapiraca - Un. Penedo')
('0', 'Ampla Concorrência')
('1', 'Reserva de Vagas - Demanda 1 (NÃO PPI, renda <= 1,5)')
('2', 'Reserva de Vagas - Demanda 2 (PPI, renda <= 1,5)')
('3', 'Reserva de Vagas - Demanda 3 (NÃO PPI, independente de renda)')
('4', 'Reserva de Vagas - Demanda 4 (PPI, independente de renda)')
('11', 'Reserva de Vagas PcD - Demanda 1 (NÃO PPI, renda <= 1,5)')
('21', 'Reserva de Vagas PcD - Demanda 2 (PPI, renda <= 1,5)')
('31', 'Reserva de Vagas PcD - Demanda 3 (NÃO PPI, independente de renda)')
('41', 'Reserva de Vagas PcD - Demanda 4 (PPI, independente de renda)')


In [11]:
df = get_relatorio_periodo('2015', 1)
df

<ipython-input-9-3356bef1e792>:11: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, item in df['Nome'].iteritems():
<ipython-input-9-3356bef1e792>:11: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, item in df['Nome'].iteritems():
<ipython-input-9-3356bef1e792>:11: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, item in df['Nome'].iteritems():
<ipython-input-9-3356bef1e792>:11: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, item in df['Nome'].iteritems():
<ipython-input-9-3356bef1e792>:11: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, item in df['Nome'].iteritems():


,Ordem,Inscrição no ENEM,Nome,Média Final no ENEM,Chamada,Classificação na Chamada,Status,Turma de Ingresso,Sexo
0,1.0,1.410429e+11,Silvio Jose Almeida De Azevedo Ramos Filho,73742.0,1ª chamada,1º,Matrícula Institucional Realizada,1º semestre,M
1,2.0,1.410184e+11,Raquel Grison,71308.0,1ª chamada,2º,Matrícula Institucional Realizada,1º semestre,F
2,3.0,1.410053e+11,Alicia Fabielle Da Costa Cavalcante,69582.0,1ª chamada,3º,Desistente - Não Entregou Documentação Da Pré-...,NaN,F
3,4.0,1.410127e+11,Mariana Cruz Santos,69422.0,1ª chamada,4º,Matrícula Institucional Realizada,1º semestre,F
4,5.0,1.410426e+11,Claudio Lucena Souza,69348.0,1ª chamada,5º,Matrícula Institucional Realizada,1º semestre,M
...,...,...,...,...,...,...,...,...,...
53,54.0,1.410690e+11,Viviane Maria Santos De Melo,41530.0,NaN,NaN,Optante Pela Lista De Espera,NaN,F
54,55.0,1.410737e+11,Kessy Mayara Barros Dos Santos,41526.0,NaN,NaN,Optante Pela Lista De Espera,NaN,F
55,56.0,1.410542e+11,Mariana Dos Santos,41138.0,NaN,NaN,Optante Pela Lista De Espera,NaN,F
56,57.0,1.410568e+11,Cleiciane Galvao Martins Da Silva,40126.0,NaN,NaN,Optante Pela Lista De Espera,NaN,F


In [16]:
df = get_relatorio_demanda('2022', 1, 4)
df

<ipython-input-9-3356bef1e792>:11: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, item in df['Nome'].iteritems():


,Ordem,Inscrição no ENEM,Nome,Média Final no ENEM,Chamada,Classificação na Chamada,Turma de Ingresso,Confirmação Matrícula,Status,Sexo
0,1.0,2.110006e+11,Bruno Henrique Rodrigues Dos Santos,61742.0,1ª chamada,1º,1º semestre,1ª conf. do 1º semestre,Matrícula Institucional realizada,M
1,2.0,2.110081e+11,Ellery Vicente Da Silva,61738.0,1ª chamada,2º,NaN,NaN,Desistente - Não entregou documentação da Pré-...,M
2,3.0,2.110124e+11,Livia Gregorio Da Silva Longo,61736.0,1ª chamada,3º,NaN,NaN,Desistente - Não entregou documentação da Pré-...,F
3,4.0,2.110366e+11,Jadiel Henrique Calado Lino,60532.0,1ª chamada,4º,NaN,NaN,Desistente - Não entregou documentação da Pré-...,M
4,5.0,2.110344e+11,Angela Cecilia De Lima Mascarenhas,59136.0,1ª chamada,5º,NaN,NaN,Desistente - Não entregou documentação da Pré-...,F
5,6.0,2.110054e+11,Jhones Eduardo Oliveira Fernandes,58694.0,1ª chamada,6º,NaN,NaN,Desistente - Não entregou documentação da Pré-...,M
6,7.0,2.110396e+11,Julio Cesar Dos Santos Tenorio,58692.0,1ª chamada,7º,NaN,NaN,Desistente - Não entregou documentação da Pré-...,M
7,8.0,2.110248e+11,Erick Tavares Ventura,58622.0,1ª chamada,8º,NaN,NaN,Desistente - Não entregou documentação da Pré-...,M
8,9.0,2.110081e+11,Vitoria Manuela Da Silva Santos,58336.0,2ª chamada,1º,1º semestre,1ª conf. do 1º semestre,Classificado na Ampla Concorrência,F
9,10.0,2.110129e+11,Keylla Rubyana Simoes Da Silva,58304.0,2ª chamada,2º,1º semestre,1ª conf. do 1º semestre,Classificado na Ampla Concorrência,F


In [13]:
'https://copeve.ufal.br/concursos/sisu_2015_acompanhamento.php'

'https://copeve.ufal.br/concursos/sisu_2015_acompanhamento.php'